In [1]:
# import our libraries
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
ticker_url = r"https://www.sec.gov/include/ticker.txt"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
response = requests.get(url = ticker_url, headers=headers)

with open('response.txt', 'wb') as f:
     f.write(response.content)
# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/include/ticker.txt


In [3]:
tickers = {}
# read text file
with open('response.txt','rb') as f:
     byte_data = f.read()
# decode data
data = byte_data.decode("utf-8").split('\n')
for line in data:
    key, value = line.split('\t')
    tickers[key] = value

In [4]:
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"
ticker = 'qdel'
param_dict = {'action':'getcompany',
              'CIK': tickers[ticker],
              'type':'10-k',
              'owner':'exclude',
              'start':'',
              'output':'atom',
              'count':'100'}


# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers = headers)
soup = BeautifulSoup(response.content, 'lxml')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=353569&type=10-k&owner=exclude&start=&output=atom&count=100


In [5]:
# find all the entry tags
entries = soup.find_all('entry')

# initalize our list for storage
master_list_xml = []
accession_num_list = []
# loop through each found entry, remember this is only the first two
for entry in entries[0:10]:
    # grab the accession number so we can create a key value
    accession_num = entry.find('accession-number').text
    accession_num_list.append(accession_num)
    # create a new dictionary
    entry_dict = {}
    entry_dict[accession_num] = {}
    
    # store the category info
    category_info = entry.find('category')    
    entry_dict[accession_num]['category'] = {}
    entry_dict[accession_num]['category']['label'] = category_info['label']
    entry_dict[accession_num]['category']['scheme'] = category_info['scheme']
    entry_dict[accession_num]['category']['term'] =  category_info['term']

    # store the file info
    entry_dict[accession_num]['file_info'] = {}
    entry_dict[accession_num]['file_info']['act'] = entry.find('act').text
    entry_dict[accession_num]['file_info']['file_number'] = entry.find('file-number').text
    entry_dict[accession_num]['file_info']['file_number_href'] = entry.find('file-number-href').text
    entry_dict[accession_num]['file_info']['filing_date'] =  entry.find('filing-date').text
    entry_dict[accession_num]['file_info']['filing_href'] = entry.find('filing-href').text
    entry_dict[accession_num]['file_info']['filing_type'] =  entry.find('filing-type').text
    entry_dict[accession_num]['file_info']['form_number'] =  entry.find('film-number').text
    entry_dict[accession_num]['file_info']['form_name'] =  entry.find('form-name').text
    entry_dict[accession_num]['file_info']['file_size'] =  entry.find('size').text
    
    # store extra info
    entry_dict[accession_num]['request_info'] = {}
    entry_dict[accession_num]['request_info']['link'] =  entry.find('link')['href']
    entry_dict[accession_num]['request_info']['title'] =  entry.find('title').text
    entry_dict[accession_num]['request_info']['last_updated'] =  entry.find('updated').text
    
    # store in the master list
    master_list_xml.append(entry_dict)
    
    print('-'*100)
    print(entry.find('form-name').text)
    print(entry.find('filing-date').text)
    print(entry.find('file-number').text)
    print(entry.find('file-number-href').text)
    print(entry.find('link')['href'])
print(accession_num_list)

----------------------------------------------------------------------------------------------------
Annual report [Section 13 and 15(d), not S-K Item 405]
2021-02-19
000-10961
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=000-10961&owner=exclude&count=100
https://www.sec.gov/Archives/edgar/data/353569/000035356921000028/0000353569-21-000028-index.htm
----------------------------------------------------------------------------------------------------
Annual report [Section 13 and 15(d), not S-K Item 405]
2020-02-13
000-10961
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=000-10961&owner=exclude&count=100
https://www.sec.gov/Archives/edgar/data/353569/000035356920000018/0000353569-20-000018-index.htm
----------------------------------------------------------------------------------------------------
Annual report [Section 13 and 15(d), not S-K Item 405]
2019-02-15
000-10961
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=000-10

In [6]:
def make_url(base_url , comp):
    url = base_url
    
    # add components of url
    for r in comp:
        url = '{}/{}'.format(url,r)
    return url

base_url = r"https://www.sec.gov/Archives/edgar/data/"
documents_url_list = []
for i in accession_num_list:
    components = [tickers[ticker], i, '.txt']
    normal_url = make_url(base_url, components)
    documents_url = normal_url.replace('-','').replace('/.txt','/index.json')
    documents_url_list.append(documents_url)
print(documents_url_list)

['https://www.sec.gov/Archives/edgar/data//353569/000035356921000028/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356920000018/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356919000013/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356918000021/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356917000012/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356916000078/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000035356915000010/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000119312514073364/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000119312513071645/index.json', 'https://www.sec.gov/Archives/edgar/data//353569/000119312512096692/index.json']


In [7]:
xml_summary_list = []
for url in documents_url_list: 
# request the url and decode it.
    content = requests.get(url,headers=headers).json()
    base_url = r"https://www.sec.gov"
    for file in content['directory']['item']:

        # Grab the filing summary andRcreate a new url leading to the file so we can download it.
        if file['name'] == 'FilingSummary.xml':

            xml_summary = base_url + content['directory']['name'] + "/" + file['name']
            xml_summary_list.append(xml_summary)

            print('-' * 100)
            print('File Name: ' + file['name'])
            print('File Path: ' + xml_summary)
print(xml_summary_list)

----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/353569/000035356921000028/FilingSummary.xml
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/353569/000035356920000018/FilingSummary.xml
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/353569/000035356919000013/FilingSummary.xml
----------------------------------------------------------------------------------------------------
File Name: FilingSummary.xml
File Path: https://www.sec.gov/Archives/edgar/data/353569/000035356918000021/FilingSummary.xml
----------------------------------------------------------------------------------------------------

In [8]:
statements_url_list = []
statements_url = []

for xml_summary in xml_summary_list:
# define a new base url that represents the filing folder. This will come in handy when we need to download the reports.
    base_url = xml_summary.replace('FilingSummary.xml', '')

    # request and parse the content
    content = requests.get(xml_summary,headers=headers).content
    soup = BeautifulSoup(content, 'lxml')

    # find the 'myreports' tag because this contains all the individual reports submitted.
    reports = soup.find('myreports')
    
    # I want a list to store all the individual components of the report, so create the master list.
    master_reports = []

    # loop through each report in the 'myreports' tag but avoid the last one as this will cause an error.
    for report in reports.find_all('report')[:-1]:

        # let's create a dictionary to store all the different parts we need.
        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['name_long'] = report.longname.text
        report_dict['url'] = base_url + report.htmlfilename.text

        # append the dictionary to the master list.
        if report_dict['name_short'] == "Selected Quarterly Financial Data (unaudited) - Quarterly Financial Data (Detail)":
        
            # print some info and store it in the statements url.
            print('-'*100)
            print(report_dict['name_short'])
            print(report_dict['url'])

            statements_url.append(report_dict['url'])
            master_reports.append(report_dict)

        # print the info to the user.


----------------------------------------------------------------------------------------------------
Selected Quarterly Financial Data (unaudited) - Quarterly Financial Data (Detail)
https://www.sec.gov/Archives/edgar/data/353569/000035356921000028/R73.htm
----------------------------------------------------------------------------------------------------
Selected Quarterly Financial Data (unaudited) - Quarterly Financial Data (Detail)
https://www.sec.gov/Archives/edgar/data/353569/000035356920000018/R73.htm
----------------------------------------------------------------------------------------------------
Selected Quarterly Financial Data (unaudited) - Quarterly Financial Data (Detail)
https://www.sec.gov/Archives/edgar/data/353569/000035356919000013/R75.htm
----------------------------------------------------------------------------------------------------
Selected Quarterly Financial Data (unaudited) - Quarterly Financial Data (Detail)
https://www.sec.gov/Archives/edgar/data/353569

In [9]:
# let's assume we want all the statements in a single data set.
statements_data = []

# loop through each statement url
for statement in statements_url:

    # define a dictionary that will store the different parts of the statement.
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    
    # request the statement file content
    content = requests.get(statement,headers=headers).content
    report_soup = BeautifulSoup(content, 'html')

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(report_soup.table.find_all('tr')):
        
        # first let's get all the elements.
        cols = row.find_all('td')
        
        # if it's a regular row and not a section or a table header
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
            
        # if it's a regular row and a section but not a table header
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['sections'].append(sec_row)
            
        # finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):            
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['headers'].append(hed_row)
            
        else:            
            print('We encountered an error.')

    # append it to the master list.
    statements_data.append(statement_data)
    print('added')

added
added
added
added
added
added
added
added


In [13]:
# Make the original Dataframe

# Grab the proper components
quarter_header =  statements_data[0]['headers'][1]
quarter_header = quarter_header[:4]
quarter_data = statements_data[0]['data']

# Put the data in a DataFrame
quarter_df = pd.DataFrame(quarter_data)

# Display
print('-'*100)
print('Before Reindexing')
print('-'*100)
display(quarter_df.head())
# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
master_index = quarter_df[0]

quarter_df = quarter_df.drop(0, axis = 1)
# Display
print('-'*100)
print('Before Regex')
print('-'*100)
display(quarter_df.head())

# Get rid of the '$', '(', ')', and convert the '' to NaNs.
quarter_df = quarter_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)

# Display
print('-'*100)
print('Before type conversion')
print('-'*100)
display(quarter_df.head())
quarter_df.drop(quarter_df.columns[4:], axis=1, inplace=True)
# drop the last columns 

# everything is a string, so let's convert all the data to a float.

quarter_df = quarter_df.astype(float)

# Change the column headers
quarter_df.columns = quarter_header

# Display
print('-'*100)
print('Final Product')
print('-'*100)

display(quarter_df)



# drop the data in a CSV file if need be.
#quarter_df.to_csv('quarter_state.csv')

----------------------------------------------------------------------------------------------------
Before Reindexing
----------------------------------------------------------------------------------------------------


,0,1,2,3,4,5,6,7,8,9,10,11
0,"Revenue from Contract with Customer, Including...","$ 809,203","$ 476,058","$ 201,754","$ 174,653","$ 152,178","$ 126,492","$ 108,252","$ 147,968","$ 1,661,668","$ 534,890","$ 522,285"
1,Cost of Goods and Services Sold,,,,,,,,,"312,813","214,085","206,572"
2,Gross profit,"701,494","383,619","148,751","114,991","94,840","75,859","59,179","90,927","1,348,855","320,805","315,713"
3,Operating income,"617,076","307,959","83,663","51,628","35,063","20,682","5,818","31,153","1,060,326","92,716","95,929"
4,Net income,"$ 470,130","$ 232,268","$ 67,652","$ 40,237","$ 30,626","$ 16,181","$ 1,270","$ 24,844","$ 810,287","$ 72,921","$ 74,183"


----------------------------------------------------------------------------------------------------
Before Regex
----------------------------------------------------------------------------------------------------


,1,2,3,4,5,6,7,8,9,10,11
0,"$ 809,203","$ 476,058","$ 201,754","$ 174,653","$ 152,178","$ 126,492","$ 108,252","$ 147,968","$ 1,661,668","$ 534,890","$ 522,285"
1,,,,,,,,,"312,813","214,085","206,572"
2,"701,494","383,619","148,751","114,991","94,840","75,859","59,179","90,927","1,348,855","320,805","315,713"
3,"617,076","307,959","83,663","51,628","35,063","20,682","5,818","31,153","1,060,326","92,716","95,929"
4,"$ 470,130","$ 232,268","$ 67,652","$ 40,237","$ 30,626","$ 16,181","$ 1,270","$ 24,844","$ 810,287","$ 72,921","$ 74,183"


----------------------------------------------------------------------------------------------------
Before type conversion
----------------------------------------------------------------------------------------------------


,1,2,3,4,5,6,7,8,9,10,11
0,809203,476058,201754,174653,152178,126492,108252,147968,1661668,534890,522285
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,312813,214085,206572
2,701494,383619,148751,114991,94840,75859,59179,90927,1348855,320805,315713
3,617076,307959,83663,51628,35063,20682,5818,31153,1060326,92716,95929
4,470130,232268,67652,40237,30626,16181,1270,24844,810287,72921,74183


----------------------------------------------------------------------------------------------------
Final Product
----------------------------------------------------------------------------------------------------


,"Dec. 31, 2020","Sep. 30, 2020","Jun. 30, 2020","Mar. 31, 2020"
0,809203.00,476058.00,201754.00,174653.00
1,NaN,NaN,NaN,NaN
2,701494.00,383619.00,148751.00,114991.00
3,617076.00,307959.00,83663.00,51628.00
4,470130.00,232268.00,67652.00,40237.00
5,11.14,5.52,1.61,0.96
6,10.78,5.33,1.55,0.93


In [14]:
# add the additonal years
for statement in statements_data[1:10]:
    # Grab the proper components
    quarter_header =  statement['headers'][1]
    quarter_header = quarter_header[:4]
    quarter_data = statement['data']

    # Put the data in a DataFrame
    year_quarter_df = pd.DataFrame(quarter_data)
#    display(year_quarter_df.head())

    # Remove all characters
    for column in year_quarter_df.columns:
        year_quarter_df[column] = year_quarter_df[column].str.replace(r'[a-zA-Z]*',"")
        year_quarter_df[column] = year_quarter_df[column].str.replace(r'-_',"")
#    display(year_quarter_df.head())

    # Display
    print('-'*100)
    print('Before Regex')
    print('-'*100)
    display(year_quarter_df.head())

    # Get rid of the '$', '(', ')', and convert the '' to NaNs.
    year_quarter_df = year_quarter_df.replace('[\$,)]','', regex=True )\
                         .replace( '[(]','-', regex=True)\
                         .replace( '', 'NaN', regex=True)

    # Display
    print('-'*100)
    print('Before type conversion')
    print('-'*100)
    display(year_quarter_df.head())

    # drop the last columns 

    # everything is a string, so let's convert all the data to a float.

    year_quarter_df.drop(year_quarter_df.columns[0], axis=1, inplace=True)
    year_quarter_df.drop(year_quarter_df.columns[4:], axis=1, inplace=True)

    
    year_quarter_df = year_quarter_df.astype(float)

    # Change the column headers
    year_quarter_df.columns = quarter_header


    # Display
    print('-'*100)
    print('Final Product')
    print('-'*100)

#    display(year_quarter_df)
    column_list = year_quarter_df.columns

    quarter_df = pd.merge(quarter_df, year_quarter_df, left_index=True, right_index = True)


In [15]:
quarter_df.index = master_index
display(quarter_df)
# Flip the dataframe
quarter_df = quarter_df.T



,"Dec. 31, 2020","Sep. 30, 2020","Jun. 30, 2020","Mar. 31, 2020","Dec. 31, 2019","Sep. 30, 2019","Jun. 30, 2019","Mar. 31, 2019","Dec. 31, 2018","Sep. 30, 2018",...,"Jun. 30, 2015","Mar. 31, 2015","Dec. 31, 2014","Sep. 30, 2014","Jun. 30, 2014","Mar. 31, 2014","Dec. 31, 2013","Sep. 30, 2013","Jun. 30, 2013","Mar. 31, 2013"
0,,,,,,,,,,,,,,,,,,,,,
"Revenue from Contract with Customer, Including Assessed Tax",809203.00,476058.00,201754.00,174653.00,152178.00,126492.00,108252.00,147968.00,132588.00,117399.00,...,35204.00,61701.00,63597.0,40857.00,31488.0,46673.00,50170.00,33539.00,29706.00,61995.00
Cost of Goods and Services Sold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15493.00,21112.00,21263.0,16768.00,15902.0,20247.00,18461.00,15297.00,13671.00,19547.00
Gross profit,701494.00,383619.00,148751.00,114991.00,94840.00,75859.00,59179.00,90927.00,82132.00,69642.00,...,18121.00,39018.00,40763.0,22518.00,14015.0,24855.00,30126.00,16695.00,14524.00,41010.00
Operating income,617076.00,307959.00,83663.00,51628.00,35063.00,20682.00,5818.00,31153.00,27538.00,16894.00,...,45029.00,53012.00,50682.0,50978.00,41473.0,48690.00,50103.00,39507.00,36829.00,44806.00
Net income,470130.00,232268.00,67652.00,40237.00,30626.00,16181.00,1270.00,24844.00,32479.00,10822.00,...,-8931.00,3991.00,7113.0,-5767.00,-6908.0,-1512.00,1139.00,-4361.00,-1755.00,12367.00
Basic and diluted loss per share,11.14,5.52,1.61,0.96,0.73,0.39,0.03,0.63,0.82,0.28,...,-0.26,0.12,0.2,-0.17,-0.2,-0.04,0.03,-0.13,-0.05,0.37
Diluted net earnings (loss) per share,10.78,5.33,1.55,0.93,0.71,0.38,0.03,0.60,0.78,0.27,...,-0.26,0.11,0.2,-0.17,-0.2,-0.04,0.03,-0.13,-0.05,0.36


In [16]:
quarter_df['Cost of Goods and Services Sold'] = quarter_df['Revenue from Contract with Customer, Including Assessed Tax'].subtract(quarter_df['Gross profit'])
quarter_df['Gross margin'] = quarter_df['Gross profit'].divide(quarter_df['Revenue from Contract with Customer, Including Assessed Tax'])
quarter_df['Operating margin'] = quarter_df['Operating income'].divide(quarter_df['Revenue from Contract with Customer, Including Assessed Tax'])
quarter_df['Profit margin'] = quarter_df['Net income'].divide(quarter_df['Revenue from Contract with Customer, Including Assessed Tax'])
quarter_df['Diluted net earnings (loss) per share'].fillna(quarter_df['Basic and diluted loss per share'], inplace = True)
display(quarter_df)
quarter_df.to_csv('clean_quarters.csv')

,"Revenue from Contract with Customer, Including Assessed Tax",Cost of Goods and Services Sold,Gross profit,Operating income,Net income,Basic and diluted loss per share,Diluted net earnings (loss) per share,Gross margin,Operating margin,Profit margin
"Dec. 31, 2020",809203.0,107709.0,701494.0,617076.0,470130.0,11.14,10.78,0.866895,0.762573,0.580979
"Sep. 30, 2020",476058.0,92439.0,383619.0,307959.0,232268.0,5.52,5.33,0.805824,0.646894,0.487899
"Jun. 30, 2020",201754.0,53003.0,148751.0,83663.0,67652.0,1.61,1.55,0.737289,0.414678,0.335319
"Mar. 31, 2020",174653.0,59662.0,114991.0,51628.0,40237.0,0.96,0.93,0.658397,0.295603,0.230383
"Dec. 31, 2019",152178.0,57338.0,94840.0,35063.0,30626.0,0.73,0.71,0.623218,0.230408,0.201251
"Sep. 30, 2019",126492.0,50633.0,75859.0,20682.0,16181.0,0.39,0.38,0.599714,0.163504,0.127921
"Jun. 30, 2019",108252.0,49073.0,59179.0,5818.0,1270.0,0.03,0.03,0.546678,0.053745,0.011732
"Mar. 31, 2019",147968.0,57041.0,90927.0,31153.0,24844.0,0.63,0.60,0.614504,0.210539,0.167901
"Dec. 31, 2018",132588.0,50456.0,82132.0,27538.0,32479.0,0.82,0.78,0.619453,0.207696,0.244962
"Sep. 30, 2018",117399.0,47757.0,69642.0,16894.0,10822.0,0.28,0.27,0.593208,0.143902,0.092181
